# MULTI-AGENT SYSTEM

This system has 2-3 agents:
- coder, this is the one that has to generate a cypher query
- executor, this is the one that test the query on a neo4j database
- retriever, this is the agent that will pass the documents to be used by coder. It cannot be considered as a part of the system because it only initiate the chat with the documents.

In [1]:
from autogen import AssistantAgent, GroupChatManager, GroupChat, config_list_from_json
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent, TEXT_FORMATS
import os

from CypherExecutor import CypherCodeExecutor

In [2]:

config_list = config_list_from_json(env_or_file="CONFIG_LIST", filter_dict={"model": "gpt-4o"})
config_list[0]["api_key"] = os.environ.get("GITHUB_TOKEN")

llm_config = {"config_list": config_list, "temperature": 0.2}

prompt = """You are a data scientist that works with Cypher queries.
Your work is to translate the given answer as a Cypher query.

You have some rules to respect:
- You must use cypher language.
- You can use the pdf files. 
- Every query must be in a separate cypher format.
- You must be precise and explain everything, code outputs too.
- You will receive the code result when executor will execute it 

You can reply with 'TERMINATE' if:
- exit code is 0 and you explained the result
- exit code is 1 and the problem can't be fixed via query

The information of the database's schema will be provided by doc_retriever in this 
context: 
{input_context}

QUESTION IS:
{input_question}
"""

print(f'Accepted formats for "docs_path": \n{TEXT_FORMATS}')

Accepted formats for "docs_path": 
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf']


In [5]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

from pathlib import Path
p = Path(__name__).parent.resolve() / "schemas"

doc_retriever = RetrieveUserProxyAgent(
    name="doc_retriever",
    is_termination_msg=termination_msg,
    max_consecutive_auto_reply=3,
    human_input_mode="NEVER",
    retrieve_config={
        "task": "code",
        "docs_path": [str(p)],
        "extra-docs": True,
        "model": config_list[0]["model"],
        "get_or_create": True,
        "customized_prompt": prompt
    },
    code_execution_config=False,
    description="Assistant who has extra content retrieval power for solving cypher queries."
)

coder      = AssistantAgent(
    name="coder",
    is_termination_msg=termination_msg,
    system_message=prompt,
    llm_config=llm_config,
)

executor   = AssistantAgent(
    name="executor",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    description= """Executor provides feedback based on the result of the query.""",
    code_execution_config={"executor": CypherCodeExecutor()}
    )


PROBLEM  = "Who acted in Pocahontas"

def _reset_agents():
    """
    This function reset all the agents used for the group chat.
    This should be used every time you start a new conversation.
    """
    doc_retriever.reset()
    coder.reset()
    executor.reset()

def state_transition(last_speaker, groupchat):
    """
    This function simply return the next speaker based on the last one.
    In this case we decided that doc_retriever should speak only to initiate chat
    and then the chat is from coder to executor in a sort of loop.
    """
    if   last_speaker is coder: return executor
    elif last_speaker is executor: return coder
    elif last_speaker is doc_retriever: return coder

def rag_chat():
    _reset_agents()
    groupchat = GroupChat(agents=[doc_retriever, coder, executor], messages=[], 
                          max_round=5, speaker_selection_method=state_transition)
    manager   = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    doc_retriever.initiate_chat(manager,message=doc_retriever.message_generator, problem=PROBLEM,n_results=3)


In [6]:
rag_chat()

2024-09-16 14:42:11,225 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.
2024-09-16 14:42:11,234 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 2 chunks.
Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Trying to create collection.
VectorDB returns doc_ids:  [['84084ab0']]
Adding content of doc 84084ab0 to context.
doc_retriever (to chat_manager):

You are a data scientist that works with Cypher queries.
Your work is to translate the given answer as a Cypher query.

You have some rules to respect:
- You must use cypher language.
- You can use the pdf files. 
- Every query must be in a separate cypher format.
- You must be precise and explain everything, code outputs too.
- You will receive the code result when executor will execute it 

You can reply with 'TERMINATE' if:
- exit code is 0 and you explained the result
- exit code is 1 and the problem can't be fixed via query

The information of the database's schema will be provided by doc_retriever in this 
context: 
[
    {
        "nodes": [
            {
                "name": "Movie",
                "indexes": [],
                "constraints": [],
                "properties": {
                    "id": [
                      

[]

executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------

Next speaker: coder

coder (to chat_manager):

The query executed successfully, but it seems there was no output. This indicates that there might not be a director recorded for the movie "Pocahontas" in the database.

Explanation:
- The query searched for a `Person` node connected to a `Movie` node with the title "Pocahontas" via the `DIRECTED` relationship.
- Since there was no result, it suggests that either the movie "Pocahontas" is not in the database, or there is no `DIRECTED` relationship for it.

If further investigation is needed, you may want to check if the movie exists in the database or if there are any directors associated with other movies.

TERMINATE

--------------------------------------------------------------------------------

Next speaker: executor

